<a href="https://colab.research.google.com/github/prof-rossetti/intro-to-python/blob/main/notebooks/applied-ds/Applied_Statistics_for_Finance_Calculating_Beta_to_the_Market_(Summer_2023).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, we will attempt to calculate a given stock's beta to the market, as one way of assessing the risk of that stock.

We will fetch some data about a number of stocks and the market.

While we have the stock and market data, we will illustrate the previously introduced statistical concept of correlation, just for fun, as we see which stocks may be positively or negatively correlated with eachother.

Then we will cover new statistical concpets of variance and covariance as we use these in our calculations for beta.

## Understanding Beta

https://www.investopedia.com/ask/answers/070615/what-formula-calculating-beta.asp


> Beta is a measure used in fundamental analysis to determine the volatility of an asset or portfolio in relation to the overall market. The overall market has a beta of 1.0, and individual stocks are ranked according to how much they deviate from the market.

> A stock that swings more than the market over time has a beta greater than 1.0. If a stock moves less than the market, the stock's beta is less than 1.0. High-beta stocks tend to be riskier but provide the potential for higher returns. Low-beta stocks pose less risk but typically yield lower returns.

> As a result, beta is often used as a risk-reward measure, meaning it helps investors determine how much risk they are willing to take to achieve the return for taking on that risk. A stock's price variability is important to consider when assessing risk. If you think of risk as the possibility of a stock losing its value, beta is useful as a proxy for risk.



> To calculate the beta of a security, the covariance between the return of the security and the return of the market must be known, as well as the variance of the market returns.



\begin{align}
        Beta = \frac{Covariance} {Variance}
\end{align}

> **Covariance** measures how two stocks move together. A positive covariance means the stocks tend to move together when their prices go up or down. A negative covariance means the stocks move opposite of each other.

> **Variance**, on the other hand, refers to how far a stock moves relative to its mean. For example, variance is used in measuring the volatility of an individual stock's price over time. Covariance is used to measure the correlation in price moves of two different stocks.

> The formula for calculating beta is the covariance of the return of an asset with the return of the benchmark, divided by the variance of the return of the benchmark over a certain period.

OK, so let's fetch some data for a handful of stocks, and then use it to illustrate statistical concepts of variance and covariance, and then we will calculate beta to the market for one or moe of the stocks.

## Fetching Stock and Market Data

Installing packages:

In [73]:
# setup cell (run and leave as is)

%%capture

!pip install yahooquery

Getting stock (and market) prices data:

In [86]:
# setup cell (run and leave as is)

# https://yahooquery.dpguthrie.com/guide/ticker/intro/
from yahooquery import Ticker

symbols = ["AAPL", "GOOGL", "META", "MSFT", "NFLX", "AMZN", "NVDA",
           "BAC", "JPM"
]
all_symbols = symbols + ["SPY"] # adding market index, but leaving the symbols variable as just a list of the individual stocks (in case we need it later)
companies = Ticker(all_symbols)
print(type(companies))

<class 'yahooquery.ticker.Ticker'>


In [87]:
from pandas import to_datetime

histories_df = companies.history()
histories_df["symbol"] = histories_df.index.get_level_values(0)
histories_df["date"] = to_datetime(histories_df.index.get_level_values(1)).date
histories_df.reset_index(drop=True, inplace=True)
print(len(histories_df)) #> 1053 rows
histories_df[["date", "symbol", "adjclose"]].head()

1220


,date,symbol,adjclose
0,2023-01-03,AAPL,124.706833
1,2023-01-04,AAPL,125.993095
2,2023-01-05,AAPL,124.656982
3,2023-01-06,AAPL,129.243622
4,2023-01-09,AAPL,129.772079


In [88]:
# quick check for null values (because some stocks may have different history lengths)
histories_df["adjclose"].isnull().sum()  #> 0 ok looks good. can proceed without concern for nulls

0

In [89]:
prices_pivot = histories_df.pivot(columns="symbol", values="adjclose", index="date")
prices_pivot.head()

symbol,AAPL,AMZN,BAC,GOOGL,JPM,META,MSFT,NFLX,NVDA,SPY
date,,,,,,,,,,
2023-01-03,124.706833,85.820000,33.030487,89.120003,133.084778,124.739998,238.460144,294.950012,143.110870,377.968140
2023-01-04,125.993095,85.139999,33.651470,88.080002,134.325806,127.370003,228.029129,309.410004,147.449707,380.886139
2023-01-05,124.656982,83.120003,33.582474,86.199997,134.296051,126.940002,221.270844,309.700012,142.611008,376.538910
2023-01-06,129.243622,86.080002,33.917610,87.339996,136.865875,130.020004,223.878601,315.549988,148.549393,385.173737
2023-01-09,129.772079,87.360001,33.405048,88.019997,136.300308,129.470001,226.058380,315.170013,156.237289,384.955383


In [90]:
print(len(prices_pivot))

earliest_date = prices_pivot.index.min()
print(earliest_date)

latest_date = prices_pivot.index.max()
print(latest_date)

122
2023-01-03
2023-06-28


Let's do some plotting of growth rates, just so we can get a better sense of the data. Also, we will use these returns as the basis for our beta calculations later.

In [91]:
import warnings
warnings.filterwarnings("ignore")

returns_df = prices_pivot.copy()
for symbol in all_symbols:
    growth_col = f"{symbol}_growth"
    # calculate relative growth :
    returns_df[growth_col] = (returns_df[symbol].pct_change(periods=1) + 1).cumprod()
    returns_df[growth_col].iloc[0] = 1

print(len(returns_df.columns))
returns_df.head()

20


symbol,AAPL,AMZN,BAC,GOOGL,JPM,META,MSFT,NFLX,NVDA,SPY,AAPL_growth,GOOGL_growth,META_growth,MSFT_growth,NFLX_growth,AMZN_growth,NVDA_growth,BAC_growth,JPM_growth,SPY_growth
date,,,,,,,,,,,,,,,,,,,,
2023-01-03,124.706833,85.820000,33.030487,89.120003,133.084778,124.739998,238.460144,294.950012,143.110870,377.968140,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2023-01-04,125.993095,85.139999,33.651470,88.080002,134.325806,127.370003,228.029129,309.410004,147.449707,380.886139,1.010314,0.988330,1.021084,0.956257,1.049025,0.992076,1.030318,1.018800,1.009325,1.007720
2023-01-05,124.656982,83.120003,33.582474,86.199997,134.296051,126.940002,221.270844,309.700012,142.611008,376.538910,0.999600,0.967235,1.017637,0.927915,1.050008,0.968539,0.996507,1.016711,1.009102,0.996219
2023-01-06,129.243622,86.080002,33.917610,87.339996,136.865875,130.020004,223.878601,315.549988,148.549393,385.173737,1.036380,0.980027,1.042328,0.938851,1.069842,1.003030,1.038002,1.026858,1.028411,1.019064
2023-01-09,129.772079,87.360001,33.405048,88.019997,136.300308,129.470001,226.058380,315.170013,156.237289,384.955383,1.040617,0.987657,1.037919,0.947992,1.068554,1.017945,1.091722,1.011340,1.024162,1.018486


In [93]:
import plotly.express as px

growth_cols = [f"{symbol}_growth" for symbol in all_symbols]

chart_df = returns_df.copy()
chart_df["date"] = chart_df.index
px.line(chart_df, x="date", y=growth_cols, title=f"Stock Returns vs Market", height=350)

## Correlation

We have previously covered correlation. While we have the stock and market data, let's take a brief detour to measure correlation between stocks.

In [95]:
cor_mat = returns_df[growth_cols].corr(method="spearman")
cor_mat

symbol,AAPL_growth,GOOGL_growth,META_growth,MSFT_growth,NFLX_growth,AMZN_growth,NVDA_growth,BAC_growth,JPM_growth,SPY_growth
symbol,,,,,,,,,,
AAPL_growth,1.000000,0.919100,0.985335,0.988801,0.555798,0.904592,0.976930,-0.636469,0.155498,0.874916
GOOGL_growth,0.919100,1.000000,0.925213,0.932410,0.663906,0.934556,0.887461,-0.560418,0.110294,0.878899
META_growth,0.985335,0.925213,1.000000,0.984750,0.531874,0.890012,0.978014,-0.648362,0.120187,0.855509
MSFT_growth,0.988801,0.932410,0.984750,1.000000,0.574032,0.908937,0.974194,-0.638799,0.131442,0.874063
NFLX_growth,0.555798,0.663906,0.531874,0.574032,1.000000,0.736172,0.494615,0.041104,0.435340,0.785201
AMZN_growth,0.904592,0.934556,0.890012,0.908937,0.736172,1.000000,0.857459,-0.445023,0.247752,0.921943
NVDA_growth,0.976930,0.887461,0.978014,0.974194,0.494615,0.857459,1.000000,-0.679644,0.108714,0.817469
BAC_growth,-0.636469,-0.560418,-0.648362,-0.638799,0.041104,-0.445023,-0.679644,1.000000,0.555489,-0.257554
JPM_growth,0.155498,0.110294,0.120187,0.131442,0.435340,0.247752,0.108714,0.555489,1.000000,0.459050


In [96]:
# https://plotly.com/python/heatmaps/
# https://plotly.com/python-api-reference/generated/plotly.express.imshow.html
import plotly.express as px

title = f"Spearman Correlation between Stock Prices (from {earliest_date} to {latest_date})"
fig = px.imshow(cor_mat,
          height=750, # title=title,
          text_auto= ".2f", #True,
          color_continuous_scale="Greens",
          color_continuous_midpoint=0
)
fig.update_layout(title={'text': title, 'x':0.485, 'xanchor': 'center'}) # https://stackoverflow.com/questions/64571789/center-plotly-title-by-default
fig.show()

What can we learn from the corrleation matrix?

Which pair or pairs of companies are most and least correlated with eachother?

Which companies are most and least correlated with the market?

If you own AAPL, which company should you consider buying if you want to hedge your risk?

## Calculating Beta

OK, correlation is fun to look at, but it is not a component of our beta calculation, so let's return our focus to calculating beta.

We saw from the "Understanding Beta" section that we need to calculcate the variance for each stock, as well as the covariance of that stock with the market.

Luckily pandas makes this easy.

### Variance

https://www.investopedia.com/terms/v/variance.asp

<img src="https://www.investopedia.com/thmb/_hIorwcVnDj-oKWhpTu_qnuUldM=/750x0/filters:no_upscale():max_bytes(150000):strip_icc():format(webp)/Variance-TAERM-ADD-Source-464952914f77460a8139dbf20e14f0c0.jpg" height=300>

> FYI: standard deviation is the square root of the variance!

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.var.html



In [97]:
returns_df.var()

symbol
AAPL             254.670828
AMZN             136.475186
BAC                8.993344
GOOGL            141.473782
JPM               24.047747
META            2076.785366
MSFT            1177.644010
NFLX            1411.690863
NVDA            5776.636046
SPY              210.117563
AAPL_growth        0.016376
GOOGL_growth       0.017813
META_growth        0.133469
MSFT_growth        0.020710
NFLX_growth        0.016227
AMZN_growth        0.018530
NVDA_growth        0.282052
BAC_growth         0.008243
JPM_growth         0.001358
SPY_growth         0.001471
dtype: float64

In [98]:
returns_df.std() ** 2 # squaring the standard deviation, for comparison

symbol
AAPL             254.670828
AMZN             136.475186
BAC                8.993344
GOOGL            141.473782
JPM               24.047747
META            2076.785366
MSFT            1177.644010
NFLX            1411.690863
NVDA            5776.636046
SPY              210.117563
AAPL_growth        0.016376
GOOGL_growth       0.017813
META_growth        0.133469
MSFT_growth        0.020710
NFLX_growth        0.016227
AMZN_growth        0.018530
NVDA_growth        0.282052
BAC_growth         0.008243
JPM_growth         0.001358
SPY_growth         0.001471
dtype: float64

### Covariance

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.cov.html

> Computes the pairwise covariance among the series of a DataFrame. The returned data frame is the covariance matrix of the columns of the DataFrame.

> This method is generally used for the analysis of time series data to understand the relationship between different measures across time.

In [99]:
cov_mat = returns_df.cov()
cov_mat

symbol,AAPL,AMZN,BAC,GOOGL,JPM,META,MSFT,NFLX,NVDA,SPY,AAPL_growth,GOOGL_growth,META_growth,MSFT_growth,NFLX_growth,AMZN_growth,NVDA_growth,BAC_growth,JPM_growth,SPY_growth
symbol,,,,,,,,,,,,,,,,,,,,
AAPL,254.670828,166.591472,-34.483249,172.053334,10.699845,716.689526,534.135184,390.081917,1147.768126,203.516299,2.042156,1.930580,5.745467,2.239935,1.322536,1.941173,8.020132,-1.043982,0.080399,0.538448
AMZN,166.591472,136.475186,-17.496344,131.062995,18.059657,473.954096,367.856273,377.256799,818.487392,157.917376,1.335865,1.470635,3.799536,1.542632,1.279053,1.590249,5.719254,-0.529703,0.135700,0.417806
BAC,-34.483249,-17.496344,8.993344,-22.705694,6.794465,-101.258976,-74.716815,-12.949879,-149.201153,-16.074809,-0.276515,-0.254777,-0.811760,-0.313330,-0.043905,-0.203873,-1.042556,0.272274,0.051054,-0.042530
GOOGL,172.053334,131.062995,-22.705694,141.473782,8.892254,494.770096,384.901626,329.749972,811.801856,148.348113,1.379662,1.587453,3.966411,1.614113,1.117986,1.527185,5.672538,-0.687416,0.066816,0.392488
JPM,10.699845,18.059657,6.794465,8.892254,24.047747,26.150989,23.578138,84.553621,64.185841,32.154771,0.085800,0.099778,0.209644,0.098877,0.286671,0.210436,0.448504,0.205703,0.180695,0.085073
META,716.689526,473.954096,-101.258976,494.770096,26.150989,2076.785366,1536.036389,1090.346139,3323.134833,565.477476,5.746995,5.551729,16.648913,6.441481,3.696715,5.522653,23.220702,-3.065622,0.196499,1.496098
MSFT,534.135184,367.856273,-74.716815,384.901626,23.578138,1536.036389,1177.644010,873.284240,2480.207940,435.263303,4.283127,4.318914,12.313904,4.938536,2.960787,4.286370,17.330675,-2.262056,0.177166,1.151587
NFLX,390.081917,377.256799,-12.949879,329.749972,84.553621,1090.346139,873.284240,1411.690863,2142.140863,476.744593,3.127992,3.700067,8.740950,3.662181,4.786204,4.395908,14.968401,-0.392058,0.635337,1.261335
NVDA,1147.768126,818.487392,-149.201153,811.801856,64.185841,3323.134833,2480.207940,2142.140863,5776.636046,963.499917,9.203731,9.109087,26.640491,10.400933,7.262725,9.537257,40.364761,-4.517074,0.482293,2.549156


If we want to calculate the covariance of "this with respect to that", we can access the specific value from this matrix. For example, the covariance of NFLX with respect to the market:

In [100]:
# if we have well defined index and columns, we can use the loc method and specify the name of the row, then the name of the column
# ... df.loc[row_name, col_name]

cov_mat.loc["NFLX", "SPY"]

476.7445929105835

### Beta

UPDATE: using the **returns** to calculate beta:

In [101]:
# calculating beta to market for a given company:
symbol = "NFLX"

# get covariance between this stock and the market
cov_mat = returns_df.cov()
cov = cov_mat.loc[symbol + "_growth", "SPY_growth"] # using loc method to access a given [row, col] combo
print(f"COVARIANCE OF {symbol} WITH RESPECT TO THE MARKET:", cov)

COVARIANCE OF NFLX WITH RESPECT TO THE MARKET: 0.0042764378481074585


In [102]:
var = returns_df["SPY_growth"].var()
print(f"VARIANCE OF THE MARKET:", var)

VARIANCE OF THE MARKET: 0.0014707944022792304


In [103]:
beta = cov / var
print(f"BETA OF {symbol} WITH RESPECT TO THE MARKET:", beta)

BETA OF NFLX WITH RESPECT TO THE MARKET: 2.9075701141372554


https://www.investopedia.com/investing/beta-gauging-price-fluctuations/

How can we interpret this beta value? What does it tell us about the company's stock, and the risk involved?